## Travel Demand Trends to Tech Hubs Using the Amadeus API


In this notebook, we focus on **data preparation**, **cleaning**, and **preprocessing** for the *Travel Demand Dataset***, a popular dataset often used for classification tasks predictinghow the AI industry boom has affected travel demand to major tech cities like San Francisco, London, Bangalore, Singapore, and Tel Aviv. It's purpose  is to build a prototype data pipeline using the Amadeus API to fetch flight demand data and analyze travel trends to these destinations, and provide recommendations for travel brands.
.

Good data preprocessing is crucial for reliable and interpretable results in machine learning and analytics workflows. Here, we address common data issues such as **missing values, duplicates, and inconsistent categorical labels** while creating derived features to improve downstream analysis.


We start by importing essential Python libraries for data handling and manipulation.

- `pandas` for structured data operations.

- `numpy` for numerical operations.

- `os` for interacting with the operating system and directory structures.



In [1]:
import pandas as pd 
import numpy as np 
import os 

## Define and Create Paths
To ensure reproducibility andorganized storage, we programmatically create directories for:

- **raw data**
- **processed data**
- **results**
- **documentation**

These directories will store intermediate and final outputs for reproducibility.


In [2]:
# Get working directory 
current_dir = os.getcwd()

# Go one directory up to the root directory 
project_root_dir = os.path.dirname(current_dir)

data_dir = os.path.join(project_root_dir, 'data')
raw_dir = os.path.join(data_dir,'raw')
processed_dir = os.path.join(data_dir,'processed')

# Define paths to results folder 
results_dir = os.path.join(project_root_dir,'results')

# Define paths to docs folder 
docs_dir = os.path.join(project_root_dir,'docs') 

#Create directories if they do not exist 
os.makedirs(raw_dir,exist_ok= True)
os.makedirs(processed_dir,exist_ok= True)
os.makedirs(results_dir,exist_ok= True)
os.makedirs(data_dir,exist_ok= True)

 ## Read in the data
 We load the **Adult Income dataset** as a CSV file.

Key considerations here are:

- We treat `?` as missing values (`na_values = '?'`).
- We use `skipinitialspace = True` to remove extra spaces after delimeters which is common in text-based datasets.

After loading, we inspect the first few rows.


In [3]:
travel_demand_filename = os.path.join(raw_dir, 'tech_city_flights.csv')
df = pd.read_csv(travel_demand_filename, na_values='?',skipinitialspace= True)
df.head(10)

,Destination,Departure Date,Airline,Price (USD),Number of Stops,Travel Time,Available Seats
0,San Francisco,7/1/2025,B6,$79.90,0,6H15M,1
1,San Francisco,7/1/2025,B6,$79.90,0,6H20M,1
2,San Francisco,7/1/2025,B6,$79.90,0,6H35M,1
3,San Francisco,7/1/2025,B6,$79.90,0,6H45M,1
4,San Francisco,7/1/2025,F9,$156.03,1,5H42M,3
5,San Francisco,7/1/2025,F9,$156.03,1,2H55M,3
6,San Francisco,7/1/2025,F9,$160.88,1,4H1M,3
7,San Francisco,7/1/2025,F9,$160.88,1,2H28M,3
8,San Francisco,7/1/2025,F9,$160.88,1,5H42M,3
9,San Francisco,7/1/2025,B6,$215.15,1,3H17M,1


We also inspect the dataset's shape. We see that the data has *10,709* rows and *7* columns.

In [4]:
df.shape

(10709, 7)

In addition, we check the data types using `.info` 

And then we adjust them according to their serving purposes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10709 entries, 0 to 10708
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Destination      10709 non-null  object
 1   Departure Date   10709 non-null  object
 2   Airline          10709 non-null  object
 3   Price (USD)      10709 non-null  object
 4   Number of Stops  10709 non-null  int64 
 5   Travel Time      10709 non-null  object
 6   Available Seats  10709 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 585.8+ KB


In [6]:
df['Departure Date'] = pd.to_datetime(df['Departure Date'])
df['Price (USD) '] = df['Price (USD) '].replace('[\$,]', '', regex=True).astype(float)


Converting Travel Time into datetime format to soften our visualisation 

In [9]:
print(df['Travel Time'].unique()[:20])


['6H15M' '6H20M' '6H35M' '6H45M' '5H42M' '2H55M' '4H1M' '2H28M' '3H17M'
 '3H' '6H28M' '1H20M' '1H28M' '1H21M' '6H40M' '6H1M' '6H21M' '6H25M'
 '6H19M' '6H']


In [11]:
# Extract hours and minutes as separate columns
time_parts = df['Travel Time'].str.upper().str.extract(r'(?P<hours>\d+)H(?P<minutes>\d*)M?')

# Fix empty minutes to zero and convert to int
time_parts['minutes'] = time_parts['minutes'].replace('', 0).astype(int)
time_parts['hours'] = time_parts['hours'].astype(int)

# Combine hours and minutes into timedelta
df['Travel Time'] = pd.to_timedelta(time_parts['hours'], unit='h') + pd.to_timedelta(time_parts['minutes'], unit='m')


In [13]:
df['Travel Time Str'] = df['Travel Time'].apply(
    lambda td: f"{int(td.total_seconds() // 3600):02d}:{int((td.total_seconds() % 3600) // 60):02d}"
)


In [14]:
print(df[['Travel Time', 'Travel Time Str']].head())


      Travel Time Travel Time Str
0 0 days 06:15:00           06:15
1 0 days 06:20:00           06:20
2 0 days 06:35:00           06:35
3 0 days 06:45:00           06:45
4 0 days 05:42:00           05:42


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10709 entries, 0 to 10708
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Destination      10709 non-null  object         
 1   Departure Date   10709 non-null  datetime64[ns] 
 2   Airline          10709 non-null  object         
 3   Price (USD)      10709 non-null  float64        
 4   Number of Stops  10709 non-null  int64          
 5   Travel Time      10709 non-null  timedelta64[ns]
 6   Available Seats  10709 non-null  int64          
 7   Travel Time Str  10709 non-null  object         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3), timedelta64[ns](1)
memory usage: 669.4+ KB


In [16]:
df['Travel Time Str'].unique

<bound method Series.unique of 0        06:15
1        06:20
2        06:35
3        06:45
4        05:42
         ...  
10704    01:48
10705    01:29
10706    07:50
10707    07:30
10708    07:55
Name: Travel Time Str, Length: 10709, dtype: object>

In [17]:
df

,Destination,Departure Date,Airline,Price (USD),Number of Stops,Travel Time,Available Seats,Travel Time Str
0,San Francisco,2025-07-01,B6,79.90,0,0 days 06:15:00,1,06:15
1,San Francisco,2025-07-01,B6,79.90,0,0 days 06:20:00,1,06:20
2,San Francisco,2025-07-01,B6,79.90,0,0 days 06:35:00,1,06:35
3,San Francisco,2025-07-01,B6,79.90,0,0 days 06:45:00,1,06:45
4,San Francisco,2025-07-01,F9,156.03,1,0 days 05:42:00,3,05:42
...,...,...,...,...,...,...,...,...
10704,Tel Aviv,2025-07-28,LY,1892.03,1,0 days 01:48:00,9,01:48
10705,Tel Aviv,2025-07-28,LY,1892.03,1,0 days 01:29:00,4,01:29
10706,Tel Aviv,2025-07-28,LX,4883.50,1,0 days 07:50:00,9,07:50
10707,Tel Aviv,2025-07-28,IB,9694.38,1,0 days 07:30:00,2,07:30


## understanding the datasets
Before proceeding with the cleaning, we would like to understanding the variables deeply. This would help guide the cleaning process. The subsequent tables detail the types, meaning and values or ranges of the variables in the dataset.

#### Data Summary and Variable Types

The dataset contains **10,709 rows** and the following columns:

**Table 1: Summary table of the variables in the dataset**

| Column          | Data Type            | Description                             |
| --------------- | -------------------- | --------------------------------------- |
| Destination     | Object (Categorical) | Name of the tech hub destination city   |
| Departure Date  | datetime64\[ns]      | Flight departure date                   |
| Airline         | Object (Categorical) | Airline operating the flight            |
| Price (USD)     | float64              | Flight price in US dollars              |
| Number of Stops | int64                | Number of stops during the flight       |
| Travel Time     | timedelta64\[ns]     | Total flight duration                   |
| Available Seats | int64                | Number of seats available on the flight |
| Travel Time Str | Object (String)      | Flight duration formatted as "HH\:MM"   |

**Table 2: Numeric Variables Table**

These are continuous or count variables useful for statistical summaries and numeric visualizations:

| Column          | Description                  | Notes                                                   |
| --------------- | ---------------------------- | ------------------------------------------------------- |
| Price (USD)     | Flight price in USD          | Used for price comparison and scatterplots              |
| Number of Stops | Number of stops on flight    | Numeric count; also treated categorically in some cases |
| Travel Time     | Flight duration as timedelta | Convert to minutes/hours for plotting and correlation   |
| Available Seats | Seats available              | Useful to analyze seat availability patterns            |

**Table 3: Categorical Variables Table**

These variables represent groups or categories for grouping and counting:

| Column      | Description      | Notes                                       |
| ----------- | ---------------- | ------------------------------------------- |
| Destination | Destination city | Key grouping for price/travel time analysis |
| Airline     | Airline name     | Grouping for airline-based insights         |

## Dealing with Missing Vlues

In [18]:
df.isnull().sum()

Destination        0
Departure Date     0
Airline            0
Price (USD)        0
Number of Stops    0
Travel Time        0
Available Seats    0
Travel Time Str    0
dtype: int64

Using `.isnull().sum()`, we identify columns with missing values

In our dataset, we have no missing values.

### 3. Removing Duplicates
Duplicates can distort statistical summaries and model performance. Using `.duplicated().sum()`, we count duplicate records.



We then inspect the duplicated records.


In [19]:
df.duplicated().sum()

2839

In [22]:
df[df.duplicated(keep=False)]

,Destination,Departure Date,Airline,Price (USD),Number of Stops,Travel Time,Available Seats,Travel Time Str
13,San Francisco,2025-07-01,B6,262.45,1,0 days 01:20:00,1,01:20
15,San Francisco,2025-07-01,B6,262.45,1,0 days 01:20:00,1,01:20
29,San Francisco,2025-07-01,AS,403.83,1,0 days 06:25:00,3,06:25
30,San Francisco,2025-07-01,AS,403.83,1,0 days 06:25:00,3,06:25
33,San Francisco,2025-07-01,AS,439.83,1,0 days 06:19:00,7,06:19
...,...,...,...,...,...,...,...,...
10694,Tel Aviv,2025-07-28,AF,1324.31,1,0 days 07:35:00,9,07:35
10697,Tel Aviv,2025-07-28,AF,1324.31,1,0 days 07:35:00,9,07:35
10700,Tel Aviv,2025-07-28,KL,1410.11,2,0 days 02:27:00,9,02:27
10701,Tel Aviv,2025-07-28,KL,1410.11,2,0 days 02:27:00,9,02:27


In [23]:
df = df.drop_duplicates()

We can confirm that we have no duplicates left in the dataset at this juncture.


In [24]:
df.duplicated().sum()

0

In [25]:
df.shape

(7870, 8)

The final shape of the clean dataset is thus *7,870* rows and *8* columns.

Finally, we save the clean, processed dataset as a CSV file in our `processed` directory for future modelling and analysis

In [26]:
final_file  = os.path.join(processed_dir,'Travel_cleaned.csv')
df.to_csv(final_file, index=False)